# Dados
Os dados dos anos de 2014 até 2017 foram disponibilizados no [kaggle](https://www.kaggle.com/schiller/cartolafc).
Dados do ano de 2018 e outros dados complementares, foram coletados do repositório [caRtola](https://github.com/henriquepgomide/caRtola/tree/master/data).


### Inconsistências

Como os dados são provenientes da API oficial do game, e esta sofreu alterações na forma em que disponibiliza os dados ao longo destes anos, então há inconsistências nos dados que impede o uso do dataset completo de forma fácil. Algumas dessas inconsistências são:

**2014:**
* Os identificadores das colunas são diferentes das tabelas dos outros anos.
* Alguns jogadores possui valores inconsistentes no campo de id do clube.

**2015:**
* Os identificadores das colunas são diferentes das tabelas dos outros anos.
* Não há informações sobre a posição do jogador.
* Não há registros da última rodada do campeonato.
* Os scouts são cumulativos, ou seja, os scouts dos jogadores vão sendo somados a cada rodada.
* Não fornece nenhum campo que identifique se o jogador participou ou não de uma rodada.

**2016:**
* Não há informações sobre a posição do jogador.

**2017 e 2018:**
* Ao contrário do que é feito nas anteriores, apresenta o nome do clube e sua abreviação, e não o identificador.
* Não fornece nenhum campo que identifique se o jogador participou ou não de uma rodada. Apenas seu status no mercado.
* Os scouts são cumulativos, ou seja, os scouts dos jogadores vão sendo somados a cada rodada.
* Os dados de 2018 estão separados em arquivos diferentes para cada rodada.

**Gerais:**
* ID de alguns clubes são diferentes em anos diferentes.
* Algumas tabelas contém valores nulos em campos importantes, como nome ou identificador do clube.

### Objetivo
O objetivo é padronizar os dados, eliminando as inconsistências citadas e resolver problemas de compatibilidade. E ainda, através desses dados extrair informações sobre as equipes do campeonato brasileiro para cada rodada de cada edição. 

In [85]:
import pandas as pd
import numpy as np
from glob import glob
from src.processing.utils import *

In [44]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [45]:
scouts14 = pd.read_csv('data/raw/2014/2014_scouts_raw.csv', low_memory=False)
scouts15 = pd.read_csv('data/raw/2015/2015_scouts_raw.csv', low_memory=False)
scouts16 = pd.read_csv('data/raw/2016/2016_scouts_raw.csv', low_memory=False)
scouts17 = pd.read_csv('data/raw/2017/2017_scouts_raw.csv', low_memory=False)

In [46]:
stock_files = sorted(glob('data/raw/2018/rodada-*.csv'))
scouts18 = pd.concat((pd.read_csv(file) for file in stock_files), ignore_index=True, sort=False)

# Padronização dos dados

O formato dos dados disponibilizados variou ao decorrer dos anos. Além dos nomes indentificadores das colunas, alguns campos foram adicionados, removidos ou tiveram seu formato alterado.

In [37]:
scouts14.columns

Index(['Atleta', 'Rodada', 'Clube', 'Participou', 'Posicao', 'Jogos', 'Pontos',
       'PontosMedia', 'Preco', 'PrecoVariacao', 'Partida', 'Mando', 'Titular',
       'Substituido', 'TempoJogado', 'Nota', 'FS', 'PE', 'A', 'FT', 'FD', 'FF',
       'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS'],
      dtype='object')

In [38]:
scouts15.columns

Index(['Rodada', 'ClubeID', 'AtletaID', 'Jogos', 'Pontos', 'PontosMedia',
       'Preco', 'PrecoVariacao', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I',
       'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS'],
      dtype='object')

In [39]:
scouts16.columns

Index(['Rodada', 'ClubeID', 'AtletaID', 'Participou', 'Pontos', 'PontosMedia',
       'Preco', 'PrecoVariacao', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I',
       'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS'],
      dtype='object')

In [40]:
scouts17.columns

Index(['Unnamed: 0', 'A', 'CA', 'CV', 'DD', 'DP', 'FC', 'FD', 'FF', 'FS', 'FT',
       'G', 'GC', 'GS', 'I', 'PE', 'PP', 'RB', 'SG', 'athletes.atletas.scout',
       'atletas.apelido', 'atletas.atleta_id', 'atletas.clube.id.full.name',
       'atletas.clube_id', 'atletas.foto', 'atletas.jogos_num',
       'atletas.media_num', 'atletas.nome', 'atletas.pontos_num',
       'atletas.posicao_id', 'atletas.preco_num', 'Rodada',
       'atletas.status_id', 'atletas.variacao_num'],
      dtype='object')

In [36]:
scouts18.columns

Index(['Unnamed: 0', 'atletas.nome', 'atletas.slug', 'atletas.apelido',
       'atletas.foto', 'atletas.atleta_id', 'atletas.rodada_id',
       'atletas.clube_id', 'atletas.posicao_id', 'atletas.status_id',
       'atletas.pontos_num', 'atletas.preco_num', 'atletas.variacao_num',
       'atletas.media_num', 'atletas.clube.id.full.name', 'FC', 'FD', 'FF',
       'FS', 'G', 'I', 'RB', 'CA', 'PE', 'A', 'SG', 'DD', 'FT', 'GS', 'CV',
       'GC', 'DP', 'PP'],
      dtype='object')

### Transformando o nome das colunas

In [47]:
dict_2014 = {'Atleta': 'atleta_id','Rodada':'rodada','Clube':'clube_id',
             'Posicao':'atleta_posicao_id','Pontos':'pontuacao',
             'PontosMedia':'pontuacao_media','Preco':'preco','PrecoVariacao':'preco_variacao'}

dict_2015 = {"Rodada":'rodada',"ClubeID":'clube_id',"AtletaID":'atleta_id',"Pontos":'pontuacao',
             "PontosMedia":'pontuacao_media',"Preco":'preco',"PrecoVariacao":'preco_variacao'}

dict_2016 = {"Rodada":'rodada',"ClubeID":'clube_id',"AtletaID":'atleta_id',"Pontos":'pontuacao',
             "PontosMedia":'pontuacao_media',"Preco":'preco',"PrecoVariacao":'preco_variacao'}

dict_2017 = {'atletas.apelido':'atleta_apelido','atletas.atleta_id':'atleta_id',
             'atletas.clube.id.full.name':'clube_nome','atletas.clube_id':'clube_id',
             'atletas.media_num':'pontuacao_media','atletas.nome':'atleta_nome',
             'atletas.pontos_num':'pontuacao','atletas.posicao_id':'posicao',
             'atletas.preco_num':'preco','Rodada':'rodada','atletas.variacao_num':'preco_variacao'}

dict_2018 = {"atletas.nome":'atleta_nome',"atletas.slug":'atleta_slug',"atletas.apelido":'atleta_apelido',
             "atletas.atleta_id":'atleta_id',"atletas.rodada_id":'rodada',"atletas.clube_id":'clube_id',
             "atletas.posicao_id":'posicao',"atletas.pontos_num":'pontuacao',"atletas.preco_num":'preco',
             "atletas.variacao_num":'preco_variacao',"atletas.media_num":'pontuacao_media',
             "atletas.clube.id.full.name":'clube_nome'}

In [48]:
scouts14.rename(columns=dict_2014, inplace=True)
scouts15.rename(columns=dict_2015, inplace=True)
scouts16.rename(columns=dict_2016, inplace=True)
scouts17.rename(columns=dict_2017, inplace=True)
scouts18.rename(columns=dict_2018, inplace=True)

In [49]:
scouts14.head()

,atleta_id,rodada,clube_id,Participou,atleta_posicao_id,Jogos,pontuacao,pontuacao_media,preco,preco_variacao,Partida,Mando,Titular,Substituido,TempoJogado,Nota,FS,PE,A,FT,FD,FF,G,I,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,Atleta,Rodada,Clube,Participou,Posicao,Jogos,Pontos,PontosMedia,Preco,PrecoVariacao,Partida,Mando,Titular,Substituido,TempoJogado,Nota,FS,PE,A,FT,FD,FF,G,I,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
1,36443,0,NaN,0,NaN,0,0,0,9,0,NaN,0,NaN,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,36443,1,285,1,1,1,5,5,10.6,1.6,179879,1,1,0,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,36443,2,285,1,1,2,-3,1,8.27,-2.33,179882,0,1,0,1,5,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,36443,3,285,1,1,3,-2.6,-0.2,6.81,-1.46,179904,1,1,0,1,6,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [50]:
scouts14 = scouts14.iloc[1:]

In [51]:
scouts15.head()

,rodada,clube_id,atleta_id,Jogos,pontuacao,pontuacao_media,preco,preco_variacao,FS,PE,A,FT,FD,FF,G,I,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,0,262,81219,0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,262,88072,0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,262,89258,0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,262,91263,0,0.0,0.0,1.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,262,74103,0,0.0,0.0,2.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
scouts16.head()

,rodada,clube_id,atleta_id,Participou,pontuacao,pontuacao_media,preco,preco_variacao,FS,PE,A,FT,FD,FF,G,I,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,0,294.0,37623,False,0.0,0.00,4.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,294.0,37623,True,5.3,5.30,6.84,2.84,0,3,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0
2,2,294.0,37623,True,-0.5,2.40,6.67,-0.17,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,3,294.0,37623,True,2.3,2.37,6.98,0.31,1,2,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0
4,4,294.0,37623,True,-2.7,1.10,5.50,-1.48,0,3,0,0,0,0,0,0,0,1,3,0,1,0,0,0,0,0


In [53]:
scouts17.head()

,Unnamed: 0,A,CA,CV,DD,DP,FC,FD,FF,FS,FT,G,GC,GS,I,PE,PP,RB,SG,athletes.atletas.scout,atleta_apelido,atleta_id,clube_nome,clube_id,atletas.foto,atletas.jogos_num,pontuacao_media,atleta_nome,pontuacao,posicao,preco,rodada,atletas.status_id,preco_variacao
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Juan,36540,Flamengo,FLA,https://s.glbimg.com/es/sde/f/2017/04/24/b3a08...,0,0.0,Juan Silveira dos Santos,0.0,zag,5.0,0,Nulo,0.0
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zé Roberto,36612,Palmeiras,PAL,https://s.glbimg.com/es/sde/f/2017/04/18/83977...,0,0.0,José Roberto da Silva Júnior,0.0,lat,8.0,0,Provável,0.0
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Paulo Autuori,36943,NaN,ATL,https://s.glbimg.com/es/sde/f/2017/04/23/51bd7...,0,0.0,Paulo Autuori,0.0,tec,10.0,0,Provável,0.0
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guto Ferreira,37245,NaN,BAH,https://s.glbimg.com/es/sde/f/2017/02/21/f5264...,0,0.0,Augusto Sérgio Ferreira,0.0,tec,4.0,0,Provável,0.0
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ney Franco,37246,NaN,SPO,https://s.glbimg.com/es/sde/f/2017/05/01/e3a7e...,0,0.0,Ney Franco da Silveira Júnior,0.0,tec,4.0,0,Provável,0.0


In [54]:
scouts18.head()

,Unnamed: 0,atleta_nome,atleta_slug,atleta_apelido,atletas.foto,atleta_id,rodada,clube_id,posicao,atletas.status_id,pontuacao,preco,preco_variacao,pontuacao_media,clube_nome,FC,FD,FF,FS,G,I,RB,CA,PE,A,SG,DD,FT,GS,CV,GC,DP,PP
0,1,Matheus Ferraz Pereira,matheus-ferraz,Matheus Ferraz,https://s.glbimg.com/es/sde/f/2018/03/17/6d461...,38632,1,AME,zag,Nulo,0.0,6.00,0.00,0.0,América-MG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Willian Lanes de Lima,lima,Lima,https://s.glbimg.com/es/sde/f/2018/03/17/3d9ef...,38506,1,AME,zag,Nulo,0.0,5.00,0.00,0.0,América-MG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Rómulo Otero Vásquez,otero,Otero,https://s.glbimg.com/es/sde/f/2017/04/03/9fe40...,83004,1,ATL,mei,Provável,16.5,14.81,9.81,16.5,Atlético-MG,1.0,2.0,2.0,2.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Diego Ribas da Cunha,diego,Diego,https://s.glbimg.com/es/sde/f/2017/08/16/3ba37...,38909,1,FLA,mei,Provável,0.8,10.89,-4.11,0.8,Flamengo,3.0,NaN,1.0,8.0,NaN,1.0,1.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Rodrigo Eduardo Costa Marinho,rodriguinho,Rodriguinho,https://s.glbimg.com/es/sde/f/2018/03/20/c125f...,61033,1,COR,mei,Provável,16.5,23.08,6.08,16.5,Corinthians,1.0,NaN,NaN,2.0,2.0,NaN,1.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Posição dos jogadores

Os dados de 2014 a 2016 não apresenta a posição do jogador, enquanto em 2017 e 2018 o atributo é representado através da abreviação. Para adicionar a informação aos anos de 2014 à 2016, é necessária a utilização de outro arquivo onde há o mapeamento de todas as posições, seus id's e as respectivas abreviações que serão utilizadas; além disso, arquivos contendo a lista de todos os jogadores que participaram em cada ano analisado. 

In [55]:
positions = pd.read_csv('data/raw/posicoes_ids.csv')
players16 = pd.read_csv('data/raw/2016/2016_jogadores.csv')
players15 = pd.read_csv('data/raw/2015/2015_jogadores.csv')
players14 = pd.read_csv('data/raw/2014/2014_jogadores.csv')

In [56]:
players14.head()

,ID,Apelido,ClubeID,PosicaoID
0,80583,Lucas Lima,277,4
1,72079,Bruno Cortez,288,2
2,86189,Mosquito,293,5
3,68938,Giovanni Augusto,316,4
4,68974,Marcão,316,5


In [57]:
def position_abbr(id, data):
    pos_id = data[data['ID'] == id]['PosicaoID'].values[0]
    pos = positions[positions['Cod'] == pos_id]['abbr'].values[0]

    return pos

In [58]:
scouts14['atleta_id'] = scouts14['atleta_id'].astype(int)
scouts15['atleta_id'] = scouts15['atleta_id'].astype(int)
scouts16['atleta_id'] = scouts16['atleta_id'].astype(int)

In [59]:
scouts14['posicao'] = scouts14.apply(lambda  row: position_abbr(row['atleta_id'], players14), axis=1)
scouts15['posicao'] = scouts15.apply(lambda  row: position_abbr(row['atleta_id'], players15), axis=1)
scouts16['posicao'] = scouts16.apply(lambda  row: position_abbr(row['atleta_id'], players16), axis=1)

### ID dos clubes

Há muitos valores nulos no campo 'clube_id' dos dados de 2014. Enquanto que os id's dos atletas estão sempre presentes. Assim é possível recuperar o identificador do clube através do id do jogador e a tabela de jogadores de 2014 que contém as duas informações.


In [60]:
scouts14['atleta_id'].isnull().sum()

0

In [61]:
scouts14['clube_id'].isnull().sum()

15229

In [62]:
def clube_id_14_16(atleta_id, data):
    return data[data['ID'] == atleta_id]['ClubeID'].values[0]

In [63]:
scouts14['clube_id'] = scouts14.apply(lambda  row: clube_id_14_16(row['atleta_id'], players14), axis=1)

Os dados de 2016 apresentam o mesmo problema

In [64]:
scouts16['clube_id'] = scouts16.apply(lambda  row: clube_id_14_16(row['atleta_id'], players16), axis=1)

Já os dados de 2017 e 2018 não possuem um valor numérico como identificador do clube, ao invés disso a abreviação e o nome são armazenados. 

* 2017 os dados apresentam diversas inconsistências, como um registro com abreviação de um clube, e o nome de outro. Desta forma, será considerado o valor do nome do clube para que então possamos substituir por seu id.

In [65]:
data = scouts17[scouts17['clube_id'] == 'ATL']
data['clube_nome'].unique()

array([nan, 'Atlético-PR', 'Atlético-MG', 'São Paulo', 'Atlético-GO',
       'Avaí', 'Coritiba', 'Ponte Preta'], dtype=object)

In [67]:
teams_2017 = pd.read_csv('data/raw/2017/2017_times.csv', sep=';')

def clube_id_2017(name):
    return teams_2017[teams_2017['Nome'] == name]['ID'].values[0]

In [68]:
scouts17 = scouts17.drop(scouts17[scouts17['clube_nome'] != scouts17['clube_nome']].index)
scouts17['clube_id'] = scouts17.apply(lambda  row: clube_id_2017(row['clube_nome']), axis=1)

* 2018 há dois clubes com a mesma abreviação.

In [69]:
data = scouts18[scouts18['clube_id'] == 'ATL']
data['clube_nome'].unique()

array(['Atlético-MG', 'Atlético-PR'], dtype=object)

In [70]:
team_dict = pd.read_csv('data/raw/times_ids.csv')

def clube_id_2018(name):
    return team_dict[team_dict['nome.cartola'] == name]['id'].values[0]

In [71]:
scouts18['clube_id'] = scouts18.apply(lambda  row: clube_id_2018(row['clube_nome']), axis=1)

Além disso, no ano de 2018 alguns clubes tiveram seus identificadores alterados. É importante converter todos os dados para corresponder aos valores do último ano para facilitar análises futuras.

In [72]:
scouts14.loc[scouts14['clube_id'] == 288, 'clube_id'] = 206
scouts15.loc[scouts15['clube_id'] == 317, 'clube_id'] = 215

### Padronizar colunas

As tabelas dos diferentes anos possuem dados que não serão úteis para a análise, como apelido do jogador, nome completo e link para sua foto. Além disso apresentam ordens diferentes nas posições das colunas.

In [73]:
columns = ['rodada', 'atleta_id', 'clube_id', 'pontuacao', 'pontuacao_media', 
           'preco', 'preco_variacao', 'posicao', 'FS', 'PE', 'A', 'FT', 'FD', 'FF', 
           'G', 'I', 'PP', 'RB', 'FC', 'GC', 'CA', 'CV', 'SG', 'DD', 'DP', 'GS']

types = {'FS': 'int64', 'PE': 'int64', 'A': 'int64', 'FT': 'int64', 'FD': 'int64', 'FF': 'int64',
         'G': 'int64', 'I': 'int64', 'PP': 'int64', 'RB': 'int64', 'FC': 'int64', 'GC': 'int64',
         'CV': 'int64', 'DD': 'int64', 'DP': 'int64', 'GS': 'int64', 'CA': 'int64', 'SG': 'int64',
         'rodada':'int32', 'clube_id': 'int32', 'pontuacao':'float32', 'pontuacao_media': 'float32',
         'preco': 'float32', 'preco': 'float32','preco_variacao':'float32'}

scouts14 = scouts14[columns].astype(types)
scouts15 = scouts15[columns].astype(types)
scouts16 = scouts16[columns].astype(types)
scouts17.fillna(0, inplace=True)
scouts17 = scouts17[columns].astype(types)
scouts18.fillna(0, inplace=True)
scouts18 = scouts18[columns].astype(types)

### Participação do jogador

Uma das informações mais importantes é sobre a participação ou não do jogador em uma determinada rodada. Apenas os dados de 2014 e 2016 registram esse campo, enquanto que em 2017 e 2018 apresenta apenas a tendência do jogador a ser ou não escalado em uma determinada rodada. Para extrair esta informação dos dados basta analisar a variação do preço do jogador e a sua pontuação na rodada, caso o valor da pontuação seja zero e não haja nenhuma variação no preço do jogador concluímos que ele não participou.

In [74]:
def played_match(row):
    score = row['pontuacao']
    variation = row['preco_variacao']

    return not(0 == score == variation)

In [75]:
scouts14['participou'] = scouts14.apply(lambda  row: played_match(row), axis=1)
scouts15['participou'] = scouts15.apply(lambda  row: played_match(row), axis=1)
scouts16['participou'] = scouts16.apply(lambda  row: played_match(row), axis=1)
scouts17['participou'] = scouts17.apply(lambda  row: played_match(row), axis=1)
scouts18['participou'] = scouts18.apply(lambda  row: played_match(row), axis=1)

### Dados cumulativos

Os dados dos anos de 2015, 2017 e 2018 são cumulativos, ou seja: os valores referentes ao número de gols marcados, passes errados, e os demais são somados a cada rodada. Essa forma de representação é diferente nos anos de 2014 e 2016, onde os valores são armazenados para cada rodada. Para manter todos os dados com uma mesma estrutura, os valores de 2014 e 2016 serão transformados em cumulativos.


In [76]:
def fill_cumulative_data(data, cols):
  
    data = data.sort_values(by=['rodada'])
    data['rodada'] = data['rodada'].astype(int)
  
    for index, row in data.iterrows():
        for column in cols:
            round = row['rodada']
            player = row['atleta_id']

            if round > 1:
                last = data[(data['atleta_id'] == player) & (data['rodada'] == round-1)]

                if len(last[column].values) > 0:
                    n = row[column]
                    m = last[column].values[0]
                    data.loc[index, column] = n + m
  
    return data

In [77]:
columns = ['FS', 'PE', 'A', 'FT', 'FD', 'FF', 'G', 'I', 'PP', 'RB', 'FC', 'GC', 
           'CA', 'CV', 'SG', 'DD', 'DP', 'GS']

scouts14 = fill_cumulative_data(scouts14, columns)
scouts16 = fill_cumulative_data(scouts16, columns)

In [79]:
scouts14['ano'] = 2014
scouts15['ano'] = 2015
scouts16['ano'] = 2016
scouts17['ano'] = 2017
scouts18['ano'] = 2018

In [84]:
scouts14.to_csv('data/processed/scouts14.csv', index=False)
scouts15.to_csv('data/processed/scouts15.csv', index=False)
scouts16.to_csv('data/processed/scouts16.csv', index=False)
scouts17.to_csv('data/processed/scouts17.csv', index=False)
scouts18.to_csv('data/processed/scouts18.csv', index=False)